In [1]:
import os
from google.cloud import pubsub_v1
import sys
import time
import pandas as pd
import http.client
import datetime
from io import StringIO

In [2]:
!python --version

Python 3.7.4


## Create connection to PubSub

In [3]:
credentials_path = 'weather-based-forecasting-v2-c4bde37656a7.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credentials_path

In [11]:
publisher_obs = pubsub_v1.PublisherClient(
    publisher_options = pubsub_v1.types.PublisherOptions(
        enable_message_ordering=True,
    )
)

publisher_forecast = pubsub_v1.PublisherClient(
    publisher_options = pubsub_v1.types.PublisherOptions(
        enable_message_ordering=True,
    )
)

topic_path_obs = 'projects/weather-based-forecasting-v2/topics/observations_topic'
topic_path_forecast = 'projects/weather-based-forecasting-v2/topics/forecasts_topic'

## Download the data

In [12]:
time.sleep(5)

#locations = ["Berlin", "Hamburg","Bremen"]

conn = http.client.HTTPSConnection("visual-crossing-weather.p.rapidapi.com")

headers = {
    "X-RapidAPI-Key": "c7b48b7c4emsh8a3b688e71b9896p1be4d0jsnda2e5b27b486",
    "X-RapidAPI-Host": "visual-crossing-weather.p.rapidapi.com"
}

conn.request("GET", "/forecast?location=Hamburg&aggregateHours=1&shortColumnNames=0&unitGroup=metric&contentType=csv",
             headers=headers)

res = conn.getresponse()
data = res.read()
s = str(data, 'utf-8')
data_string = StringIO(s)
df = pd.read_csv(data_string)
# df.to_csv("./weather_prediction_hamburg.csv")

In [13]:
df2 = df.copy()
df2.index = pd.to_datetime(df2["Date time"])
df2 = df2.asfreq(freq="1min")
df2 = df2.interpolate(method ='linear', limit_direction ='forward')
df2["Conditions"] = df2.Conditions.fillna(method="backfill")
df2["Address"] = df2.Address[0]
df2["Date time"] = df2.index
df2["Resolved Address"] = df2.Name[0]
df2["Name"] = df2.Name[0]

# we're taking 10 days of a forecast
# now = datetime.datetime.now()
# minutes = now.minute
# df3 = df2.iloc[minutes-1:60*24*10, :]
df3 = df2.iloc[:60*24*10, :]
#df3.to_csv(sys.stdout, index=False)

In [14]:
now = datetime.datetime.now()
minutes = now.minute
df3_hour = df3.iloc[minutes-1:minutes+59, :]
df3_hour.head(5)

,Address,Date time,Latitude,Longitude,Resolved Address,Name,Wind Direction,Temperature,Wind Speed,Cloud Cover,Heat Index,Chance Precipitation (%),Precipitation,Sea Level Pressure,Snow Depth,Snow,Relative Humidity,Wind Gust,Wind Chill,Conditions
Date time,,,,,,,,,,,,,,,,,,,,
2022-12-18 16:50:00,Hamburg,2022-12-18 16:50:00,53.5537,9.99183,Hamburg,Hamburg,122.983333,-4.30,11.783333,100.0,NaN,0.0,0.0,1028.166667,0.0,0.0,77.933333,26.00,-8.950,Overcast
2022-12-18 16:51:00,Hamburg,2022-12-18 16:51:00,53.5537,9.99183,Hamburg,Hamburg,122.985000,-4.31,11.795000,100.0,NaN,0.0,0.0,1028.150000,0.0,0.0,77.970000,26.03,-8.965,Overcast
2022-12-18 16:52:00,Hamburg,2022-12-18 16:52:00,53.5537,9.99183,Hamburg,Hamburg,122.986667,-4.32,11.806667,100.0,NaN,0.0,0.0,1028.133333,0.0,0.0,78.006667,26.06,-8.980,Overcast
2022-12-18 16:53:00,Hamburg,2022-12-18 16:53:00,53.5537,9.99183,Hamburg,Hamburg,122.988333,-4.33,11.818333,100.0,NaN,0.0,0.0,1028.116667,0.0,0.0,78.043333,26.09,-8.995,Overcast
2022-12-18 16:54:00,Hamburg,2022-12-18 16:54:00,53.5537,9.99183,Hamburg,Hamburg,122.990000,-4.34,11.830000,100.0,NaN,0.0,0.0,1028.100000,0.0,0.0,78.080000,26.12,-9.010,Overcast


## Send observations in intervals to PubSub - check Weather_PubSub_check for results

In [15]:
for i in range(3):
    obs = str(df3_hour.iloc[i,:].astype(str).to_dict())
    obs = obs.replace("'", '"')
    index = df3_hour.iloc[i,:].name
    index = str(int(index.value / 10**9)).encode("utf-8")
    obs = obs.encode("utf-8")
    future = publisher_obs.publish(topic_path_obs, obs, ordering_key=index)
    print(f'published message id {future.result()}')
    time.sleep(60)

published message id 6606510385934138
published message id 6606507670839124
published message id 6606511062031710


## Send Forecast in intervals to PubSub

In [16]:
df3_forecast = df3.iloc[minutes-1:, :]
df3_forecast.index = df3_forecast.index.astype(str)
for i in range(60):
    
    df3_forecast_window = str(df3_forecast.iloc[i:i+7*24*60,:].astype(str).to_dict("index"))
#     for key in df3_forecast_window.keys():
#         df3_forecast_window[key] = str(df3_forecast_window[key])
    df3_forecast_window = df3_forecast_window.encode('utf-8')
        
    info = "Weather forecast for Hamburg, {}".format(df3_forecast.index[i])
    info = info.encode('utf-8')
    
    future = publisher_forecast.publish(topic_path_forecast, df3_forecast_window)
    print(f'published message id {future.result()}')
    time.sleep(60)